# TESTING A DIFFERENT SOLVER

In [1]:
%%capture
import sys
import os

if 'google.colab' in sys.modules:
    !pip install idaes-pse --pre
    !idaes get-extensions --to ./bin
    os.environ['PATH'] += ':bin'

In [4]:
import pandas as pd

cost = pd.read_csv("cost_vector.csv", index_col= 0 )
ebitda = pd.read_csv("ebitda_matrix.csv",index_col= 0)


In [5]:
import pyomo.environ as pyo

In [7]:
model = pyo.ConcreteModel()

num_y = cost.size

num_rows = ebitda.shape[0]
num_cols = ebitda.shape[1]

# Create x
model.rows = pyo.RangeSet(1,num_rows)
model.cols = pyo.RangeSet(1,num_cols)

# Define decision variables
model.x = pyo.Var(model.rows, model.cols, within = pyo.Binary)
model.y = pyo.Var(range(1, num_y + 1), within = pyo.Binary)

# Define the objective function programmatically
def profit_function(model):
    # Calculate total cost
    cost_total = sum(model.y[j] * cost.iloc[j - 1, 0] for j in range(1, num_y + 1))

    # Calculate total EBITDA
    ebitda_total = sum(model.y[j] * ebitda.iloc[i - 1, j - 1] * model.x[i, j] for i in model.rows for j in range(1, num_y + 1))

    # Calculate profit
    profit_total = ebitda_total - cost_total
    return profit_total

model.obj = pyo.Objective(rule = profit_function, sense = pyo.maximize)

# Define constraints
model.constraints = pyo.ConstraintList()
for i in range(1,num_rows):
    model.constraints.add(sum(model.x[i,j] for j in range(1,num_cols)) <= 1)

def y_constraint_rule(model, i, j):
    return model.x[i, j] <= model.y[j]

model.y_constraint = pyo.Constraint(model.rows, model.cols, rule=y_constraint_rule)


In [11]:
solver = pyo.SolverFactory('ipopt')
# solver = pyo.SolverFactory('couenne') #one of these two algorithms
results = solver.solve(model, tee = True)

ipopt


ApplicationError: No executable found for solver 'ipopt'

In [20]:
ebitda.size + cost.size

113025